In [1]:
# Inception

In [2]:
from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
# from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import applications
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

Using TensorFlow backend.


In [3]:
from sklearn.metrics import accuracy_score
# organize imports
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [4]:
img_width, img_height = 128, 128
train_data_dir = "data/MSTAR_Chips/TRAIN_images"
validation_data_dir = "data/MSTAR_Chips/TEST_images"
nb_train_samples = 1622
nb_validation_samples = 1365 
batch_size = 32
epochs = 50

In [5]:
img_width, img_height = 224,224

In [8]:
model = applications.InceptionV3(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 111, 111, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 111, 111, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 111, 111, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [11]:
feat_ext1=Model(input=model.input, output=model.get_layer('mixed10').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mi...)`
  if __name__ == '__main__':


In [13]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
inception_features = []
inception_test_features = []
inception_labels   = []
inception_test_labels =[]
image_size=(224,224)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		inception_features.append(flat)
		inception_labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		inception_test_features.append(flat)
		inception_test_labels.append(label)

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [14]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
inception_le_labels = le.fit_transform(inception_labels)
inception_test_labels=le.transform(inception_test_labels)

In [15]:
seed=2017

In [16]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(inception_features, inception_le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
# evaluate the model of test data
preds = model.predict(inception_test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(inception_test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(inception_test_labels)))

                 precision    recall  f1-score   support

      bmp2_tank       0.84      0.90      0.87       549
btr70_transport       0.78      0.83      0.80       183
       t72_tank       0.95      0.88      0.91       633

    avg / total       0.88      0.88      0.88      1365

Accuracy : 0.878388278388


## VGG16

In [18]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [19]:
feat_ext_vgg=Model(input=model_.input, output=model_.get_layer('block5_pool').output)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  if __name__ == '__main__':


In [20]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold features and labels
vgg_features = []
vgg_test_features = []
vgg_labels   = []
vgg_test_labels =[]
image_size=(128,128)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext_vgg.predict(x)
		flat = feature.flatten()
		vgg_features.append(flat)
		vgg_labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext_vgg.predict(x)
		flat = feature.flatten()
		vgg_test_features.append(flat)
		vgg_test_labels.append(label)
		

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [21]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
vgg_le_labels = le.fit_transform(vgg_labels)
vgg_test_labels=le.transform(vgg_test_labels)

In [22]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(vgg_features, vgg_le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
# evaluate the model of test data
preds = model.predict(vgg_test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(vgg_test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(vgg_test_labels)))

                 precision    recall  f1-score   support

      bmp2_tank       0.87      0.92      0.89       550
btr70_transport       0.82      0.82      0.82       196
       t72_tank       0.97      0.91      0.94       619

    avg / total       0.91      0.90      0.90      1365

Accuracy : 0.903296703297


1622

## Combine Analysis

In [4]:
img_width, img_height = 224,224
train_data_dir = "data/MSTAR_Chips/TRAIN_images"
validation_data_dir = "data/MSTAR_Chips/TEST_images"
nb_train_samples = 1622
nb_validation_samples = 1365 
batch_size = 32
epochs = 50

In [7]:
model = applications.InceptionV3(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
feat_ext1=Model(input=model.input, output=model.get_layer('mixed10').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("mi..., inputs=Tensor("in...)`
  


In [6]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
feat_ext_vgg=Model(input=model_.input, output=model_.get_layer('block5_pool').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  


In [7]:
sq_model =applications.Xception(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [10]:
sq_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 111, 111, 32)  864         input_3[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 111, 111, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 111, 111, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [8]:
feat_ext_sq=Model(input=sq_model.input, output=sq_model.get_layer('block14_sepconv2_act').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  """Entry point for launching an IPython kernel.


In [9]:
# path to training dataset
train_labels = os.listdir(train_data_dir)


# variables to hold inception features and labels
inception_features = []
inception_test_features = []

# variables to hold vgg features and labels
vgg_features = []
vgg_test_features = []

# variables to hold vgg features and labels
sq_features = []
sq_test_features = []


labels   = []
test_labels =[]

image_size=(224,224)
# loop over all the labels in the folder
for label in train_labels:
	cur_path = train_data_dir + "/" + label
	print(cur_path)
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		inception_features.append(flat)
		feature = feat_ext_vgg.predict(x)
		flat = feature.flatten()
		vgg_features.append(flat)
		feature = feat_ext_sq.predict(x)
		flat = feature.flatten()
		sq_features.append(flat)
		labels.append(label)

# loop over all the labels in the folder
for label in train_labels:
	cur_path = validation_data_dir + "/" + label
	for image_path in glob.glob(cur_path + "/*.jpg"):
		img = image.load_img(image_path, target_size=image_size)
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		feature = feat_ext1.predict(x)
		flat = feature.flatten()
		inception_test_features.append(flat)
		feature = feat_ext_vgg.predict(x)
		flat = feature.flatten()
		vgg_test_features.append(flat)
		feature = feat_ext_sq.predict(x)
		flat = feature.flatten()
		sq_test_features.append(flat)
		test_labels.append(label)

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [10]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [11]:
np.asarray(vgg_features).shape

(1622, 25088)

In [12]:
np.asarray(inception_features).shape

(1622, 51200)

In [13]:
np.asarray(sq_features).shape

(1622, 100352)

In [14]:
features = np.concatenate( [np.asarray(vgg_features), np.asarray(inception_features), np.asarray(sq_features)], axis=1 )

In [15]:
test_features = np.concatenate( [np.asarray(vgg_test_features), np.asarray(inception_test_features),np.asarray(sq_test_features)], axis=1 )

In [16]:
seed=2017

In [17]:
import xgboost as xgb

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# Prepare the inputs for the model
train_X = np.asmatrix(features)
test_X = np.asmatrix(test_features)
#train_y = train_df['Survived']

# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, le_labels)
predictions = gbm.predict(test_X)

In [64]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [65]:
# evaluate the model of test data
preds = model.predict(test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))

                 precision    recall  f1-score   support

      bmp2_tank       0.87      0.92      0.90       555
btr70_transport       0.79      0.85      0.82       181
       t72_tank       0.97      0.90      0.94       629

    avg / total       0.91      0.90      0.91      1365

Accuracy : 0.904029304029


In [66]:
from sklearn.decomposition import PCA

In [67]:
features.shape

(1622, 176640)

In [68]:
test_features.shape

(1365, 176640)

In [69]:
X=np.concatenate( [features,test_features], axis=0 )

In [70]:
X.shape

(2987, 176640)

In [71]:
pca = PCA(n_components=10000)

In [72]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=10000, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [73]:
# use logistic regression as the model
print("[INFO] creating model...")
clf = LogisticRegression(random_state=seed)
clf.fit(pca.transform(features), le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
# evaluate the model of test data
preds = clf.predict(pca.transform(test_features))
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))